# Data preparison 

Take the input data from Stula/Stefan and put them into the required format for the simulation scripts.

### revision 19.10,2022,

Adapted the code so that it works on DAPLA.


### revision 24, may, 2022:

message from Sturla:

Ive generated a new matrix. replaced the mov_mat_long.csv with the variables year, agesexgr, knr_origin, knr_destination, prob. do you only use the long input files? If so, I wont bother generating the wide version

I also remade the mov_mat_link.csv, which now is only a link from ( age, sex ) -> agesexgr

### Region recoding
Regions are coded with [0,1,2...,355] not the actual knr. 

### origingr recoding
Origingr is coded with [0,20] not the original one

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spo
import pandas as pd
import subprocess
import os 
from itertools import compress
import copy
import pickle
import math
import sim_para
import time
from dapla import FileClient

total number of available CPUs is 80


In [2]:
#path=os.path.expanduser('~/Population/g2022')

# base_year=2022
# maxage=120
# range_fertility=49-15+1
# max_year=31
# number_region=356
# move_age=69

path='ssb-prod-forsk-sim-befram-data-synk-opp/rawdata'
workpath = sim_para.workpath
max_year=sim_para.max_year
base_year=sim_para.base_year
move_age=sim_para.move_age
number_region=sim_para.number_region
maxage=sim_para.maxage
range_fertility=sim_para.range_fertility
start_year=sim_para.start_year
move_max=sim_para.move_max

print("base_year is set to:", base_year)
print("fertility rate for first year projection used is for year :", start_year)
print("path is set to:", path)
print("work path is set to:", workpath)

base_year is set to: 2021
fertility rate for first year projection used is for year : 2022
path is set to: ssb-prod-forsk-sim-befram-data-synk-opp/rawdata
work path is set to: ssb-prod-forsk-sim-befram-data-produkt


In [3]:
datafile=path+'/'+'population.csv'
#data_population=pd.read_csv(datafile)
data_population = FileClient.load_csv_to_pandas(f"{datafile}") 
data_population.to_csv('population.csv', index=False)

In [4]:


## generate a link between knr and our region

a=data_population['knr'].unique()
a.sort()

df_knr=pd.DataFrame(a)

df_knr.reset_index(inplace=True)

df_knr.head()

df_knr.columns=['region', 'knr' ]

df_knr.to_csv('df_knr.csv', index=False)

# if knr == 0 immigrants, then region == number_region

df2 = {'region': number_region, 'knr': 0}
df_knr = df_knr.append(df2, ignore_index = True)

print(df_knr.head())

## merge the region variable in

df_pop=data_population.merge(df_knr, on='knr')[data_population['year']==start_year]

df_pop.drop(columns=['knr', 'year'], inplace=True)

outfile='population_county.csv'

df_pop.to_csv('population_county.csv', index=False)
#FileClient.save_pandas_to_csv(df_pop, f"{outfile}")

print(df_pop.head())

df_knr.tail()

/tmp/ipykernel_433/3591672733.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_knr = df_knr.append(df2, ignore_index = True)


   region   knr
0       0   301
1       1  1101
2       2  1103
3       3  1106
4       4  1108
      age  sex   pop  region
2880    0    1  4580       0
2881    0    2  4557       0
2882    1    1  4097       0
2883    1    2  4022       0
2884    2    1  4008       0


,region,knr
352,352,5441
353,353,5442
354,354,5443
355,355,5444
356,356,0


In [5]:
datafile='.'+'/'+'population_county.csv'
df_pop=pd.read_csv(datafile)
#df_pop=FileClient.load_csv_to_pandas(f"{datafile}") 
df_pop.head()

,age,sex,pop,region
0,0,1,4580,0
1,0,2,4557,0
2,1,1,4097,0
3,1,2,4022,0
4,2,1,4008,0


In [6]:
df_knr

,region,knr
0,0,301
1,1,1101
2,2,1103
3,3,1106
4,4,1108
...,...,...
352,352,5441
353,353,5442
354,354,5443
355,355,5444


In [7]:
datafile=path+'/'+'fertility_long.csv'
data_fertility=FileClient.load_csv_to_pandas(f"{datafile}") 


## merge the region variable in

df_temp=data_fertility.merge(df_knr, on='knr')

df_temp.drop(columns=['knr'], inplace=True)

df_temp.to_csv('fertility.csv', index=False)
## year specific fertility rates
## need some changes in codes


In [8]:
datafile='fertility.csv'
df_temp=pd.read_csv(datafile)
#FileClient.load_csv_to_pandas(f"{datafile}") 
df_temp

,year,age,prob,region
0,2022,15,0.000025,0
1,2022,16,0.000216,0
2,2022,17,0.000398,0
3,2022,18,0.001099,0
4,2022,19,0.002153,0
...,...,...,...,...
386255,2052,45,0.002015,355
386256,2052,46,0.000258,355
386257,2052,47,0.000308,355
386258,2052,48,0.000129,355


In [9]:
## test fertility rate input



fertility_table=[np.zeros((number_region,range_fertility)) for i in range(max_year)]

print(fertility_table[0][0,0])

datafile='fertility.csv'
data_in=pd.read_csv(datafile)
#FileClient.load_csv_to_pandas(f"{datafile}")  
        #print(data_in)
for index, row in data_in.iterrows():
    index_temp=int(row.year-start_year)
    # region: 0 ... 355
    r=int(row.region)
    #age start at 15
    a=int(row.age)-15
    fertility_table[index_temp][r,a]=row.prob 



0.0


In [10]:
data_in.loc[ (data_in['year'] == 2022) & (data_in['region']==0)]

,year,age,prob,region
0,2022,15,0.000025,0
1,2022,16,0.000216,0
2,2022,17,0.000398,0
3,2022,18,0.001099,0
4,2022,19,0.002153,0
5,2022,20,0.003572,0
6,2022,21,0.005442,0
7,2022,22,0.008134,0
8,2022,23,0.011561,0
9,2022,24,0.015476,0


In [11]:
fertility_table[0][0,:]

array([2.4732703e-05, 2.1638478e-04, 3.9782829e-04, 1.0988283e-03,
       2.1526695e-03, 3.5723117e-03, 5.4418873e-03, 8.1337998e-03,
       1.1561136e-02, 1.5475845e-02, 2.0848246e-02, 2.9297085e-02,
       3.9620932e-02, 5.3175658e-02, 7.0838042e-02, 9.2512794e-02,
       1.1177602e-01, 1.2445483e-01, 1.2716419e-01, 1.2188685e-01,
       1.1374494e-01, 1.0226244e-01, 8.6754449e-02, 7.0251480e-02,
       5.7793979e-02, 4.4767868e-02, 3.2466553e-02, 2.2250842e-02,
       1.4174573e-02, 7.4357609e-03, 4.1969577e-03, 2.7302878e-03,
       1.7342080e-03, 1.0646691e-03, 4.2660444e-04])

In [12]:
datafile=path+'/'+'mortality_long.csv'
data_mortality=FileClient.load_csv_to_pandas(f"{datafile}") 

## year specific mortality rates
## merge the region variable in

df_temp=data_mortality.merge(df_knr, on='knr')

df_temp.drop(columns=['knr'], inplace=True)

# df_temp.loc[df_temp.age == maxage, ['prob']] = 1.0
df_temp.to_csv('mortality.csv', index=False)

datafile='mortality.csv'
df_temp=pd.read_csv(datafile)
#FileClient.load_csv_to_pandas(f"{datafile}") 
df_temp

,year,age,sex,prob,region
0,2022,0,1,0.001179,0
1,2022,0,2,0.001359,0
2,2022,1,1,0.000327,0
3,2022,1,2,0.000200,0
4,2022,2,1,0.000030,0
...,...,...,...,...,...
2477755,2050,117,2,0.500000,355
2477756,2050,118,1,0.500000,355
2477757,2050,118,2,0.500000,355
2477758,2050,119,1,0.500000,355


In [13]:
mortality_table=[np.ones((number_region, 2, maxage+1)) for i in range(max_year)]
data_in=df_temp
for index, row in data_in.iterrows():
    index_temp=int(row.year-start_year)
    # region: 1-366
    r=int(row.region)
    # sex=1 or 2
    s=int(row.sex)-1
    #age 0 to 119
    a=int(row.age)
    mortality_table[index_temp][r,s,a]=float(row.prob)


In [14]:
mortality_table[29][0,0,1]

1.0

In [15]:
datafile=path+'/'+'migration_long.csv'
data_migration=FileClient.load_csv_to_pandas(f"{datafile}") 


df_temp=data_migration.merge(df_knr, on='knr')

df_temp.drop(columns=['knr'], inplace=True)

# df_temp.loc[df_temp.age == maxage, ['prob']] = 1.0
testpath='.'
datafile=testpath+'/'+'migration.csv'
df_temp.to_csv(datafile, index=False)



datafile='.'+'/'+'migration.csv'
df_temp=pd.read_csv(datafile)
#FileClient.load_csv_to_pandas(f"{datafile}") 
df_temp
# pr_mig is the same across year? 

,year,age,sex,pr_mov,pr_mig,region
0,2022,0,1,0.085187,0.011570,0
1,2022,0,2,0.083389,0.010594,0
2,2022,1,1,0.085349,0.013471,0
3,2022,1,2,0.084010,0.012309,0
4,2022,2,1,0.078604,0.014580,0
...,...,...,...,...,...,...
1545035,2052,67,2,0.008149,0.001786,355
1545036,2052,68,1,0.007110,0.002157,355
1545037,2052,68,2,0.007682,0.001585,355
1545038,2052,69,1,0.006331,0.001940,355


In [16]:
outmigration_table=[np.zeros((number_region, 2, move_age+1)) for i in range(max_year)]
move_table=[np.zeros((number_region, 2, move_age+1)) for i in range(max_year)]
data_in=df_temp
for index, row in data_in.iterrows():
    index_temp=int(row.year-start_year)
    # region: 1-366
    r=int(row.region)
    # sex=1 or 2
    s=int(row.sex)-1
    #age 0 to 119
    a=int(row.age)
    outmigration_table[index_temp][r,s,a]=float(row.pr_mig)
    move_table[index_temp][r,s,a]=float(row.pr_mov)

In [17]:
datafile=path+'/'+'distr_immigration.csv'
data_dist_imm=FileClient.load_csv_to_pandas(f"{datafile}") 
data_dist_imm.to_csv('distr_immigration.csv', index=False)

print(data_dist_imm.loc[(data_dist_imm['year']==2022)&(data_dist_imm['sex']==2)])

# note that one cannot use list multiplication to initlize the list
# if you change one elements, all will be changed. 

im_dist_table=[np.zeros(((move_age+1)*2)) for i in range(max_year)]

for index, row in data_dist_imm.iterrows():
    index_temp=int(row.year-start_year)
    # sex=1 or 2
    s=int(row.sex)-1
    #age 0 to 119
    a=int(row.age)
    # print(index_temp, s,a, row.pr_immigrant)
    im_dist_table[index_temp][s*(move_age+1)+a]=float(row.pr_immigrant)
    

print(im_dist_table[0][:].sum())
print(im_dist_table[0][:])
print(im_dist_table[0][69])
print(im_dist_table[0][139])

     year  age  sex  pr_immigrant
1    2022    0    2      0.004027
3    2022    1    2      0.009506
5    2022    2    2      0.008163
7    2022    3    2      0.007456
9    2022    4    2      0.006893
..    ...  ...  ...           ...
131  2022   65    2      0.000853
133  2022   66    2      0.000816
135  2022   67    2      0.000707
137  2022   68    2      0.000726
139  2022   69    2      0.000617

[70 rows x 4 columns]
1.0000000000000004
[0.00428118 0.01003175 0.00856236 0.00778231 0.0072381  0.00714739
 0.0067483  0.00622222 0.00611338 0.00578685 0.005678   0.00551474
 0.00517007 0.0054966  0.0054966  0.00595011 0.00756463 0.00821769
 0.00689342 0.00653061 0.00758277 0.0093424  0.01092063 0.01364172
 0.01612698 0.0185941  0.01986395 0.01990023 0.0195737  0.01924717
 0.0185941  0.01728798 0.01623583 0.01503855 0.01378685 0.01275283
 0.01197279 0.01097506 0.00999546 0.00966893 0.00878005 0.00781859
 0.00729252 0.00680272 0.00611338 0.00571429 0.00544218 0.00487982
 0.00440816 0.

In [18]:
datafile=path+'/'+'tot_migration.csv'
data_tot_mig=FileClient.load_csv_to_pandas(f"{datafile}") 
data_tot_mig.to_csv('tot_migration.csv', index=False)
data_tot_mig.head(20)

## round? how? nearest interger

,year,tot_immigrants,tot_emigrants
0,2022,67125.820,29874.795
1,2023,55122.246,30958.881
2,2024,43309.129,31489.813
3,2025,42894.340,30619.359
4,2026,42831.805,30141.605
5,2027,42683.387,29759.775
6,2028,42462.684,29258.359
7,2029,42198.238,28972.711
8,2030,41919.637,28861.566
9,2031,41650.832,28831.840


In [19]:
datafile=path+'/'+'mov_mat_long.csv'
data_mov_mat=FileClient.load_csv_to_pandas(f"{datafile}") 

data_mov_mat.rename(columns={'knr_destination':'knr'}, inplace=True)

df_temp=data_mov_mat.merge(df_knr, on='knr')

df_temp.drop(columns=['knr'], inplace=True)

df_temp.rename(columns={'knr_origin':'knr', 'region':'d_region'}, inplace=True)

df_temp2=df_temp.merge(df_knr, on='knr')

df_temp2.drop(columns=['knr'], inplace=True)

df_temp3=df_temp2[df_temp2['year']<=move_max]

# df_temp.loc[df_temp.age == maxage, ['prob']] = 1.0
df_temp3.to_csv('mov_mat.csv', index=False)

df_temp3.head()

,year,agesexgr,prob,d_region,region
0,2022,1,0.192298,0,356
1,2022,2,0.157400,0,356
2,2022,3,0.180167,0,356
3,2022,4,0.211331,0,356
4,2022,5,0.237535,0,356


In [20]:
datafile='mov_mat.csv'
df_mov_mat=pd.read_csv(datafile)
#FileClient.load_csv_to_pandas(f"{datafile}") 

df_mov_mat.head()
print(df_mov_mat.head())
mov_mat=[np.zeros((number_region+1,20,number_region)) for i in range(move_max-start_year+1)]

print(time.time())

for index, row in df_mov_mat.iterrows():
    index_temp=int(row.year-start_year)
    # sex=1 or 2
    s=int(row.agesexgr)-1
    #age 0 to 119
    a=int(row.region)
    #age 0 to 119
    r=int(row.d_region)
    mov_mat[index_temp][a,s,r]=float(row.prob)
    
print(time.time())

   year  agesexgr      prob  d_region  region
0  2022         1  0.192298         0     356
1  2022         2  0.157400         0     356
2  2022         3  0.180167         0     356
3  2022         4  0.211331         0     356
4  2022         5  0.237535         0     356
1666382129.901637
1666382905.750537


In [21]:
# check that all probability for a given region sum up to 1
for i in range(move_max-start_year+1):
    print(mov_mat[i][0,0,:].sum())


0.9999999999999993
0.9999999999999988
1.0000000000000022
0.999999999999998
1.0000000000000013
1.0000000000000007


In [22]:
datafile=path+'/'+'mov_mat_link.csv'
data_link=FileClient.load_csv_to_pandas(f"{datafile}") 
data_link_a=data_link[['age', 'sex', 'agesexgr']].copy()
data_link_a.drop_duplicates(subset=None, keep='first', inplace=True)
data_link_a.to_csv('link_a.csv', index=False)

##### no longer needed 

datafile=path+'/'+'mov_mat_link.csv'
data_link=FileClient.load_csv_to_pandas(f"{datafile}") 
data_link_r=data_link[['knr', 'origingr']].copy()
data_link_r.drop_duplicates(subset=None, keep='first', inplace=True)
data_temp=data_link_r.merge(df_knr, on='knr')
data_link_r=data_temp.merge(df_origingr, on='origingr')
data_link_r.drop(columns=['origingr', 'knr'], inplace=True)
data_link_r.to_csv(path+'/'+'link_r.csv', index=False)

In [23]:
datafile='link_a.csv'
data_link_a=pd.read_csv(datafile)
#data_link_a=FileClient.load_csv_to_pandas(f"{datafile}") 
print(data_link_a.loc[data_link_a['agesexgr']==20])
print(data_link_a.loc[data_link_a['agesexgr']==18])
data_link_a.tail(20)

     age  sex  agesexgr
105   52    2        20
107   53    2        20
109   54    2        20
111   55    2        20
113   56    2        20
115   57    2        20
117   58    2        20
119   59    2        20
121   60    2        20
123   61    2        20
125   62    2        20
127   63    2        20
129   64    2        20
131   65    2        20
133   66    2        20
135   67    2        20
137   68    2        20
139   69    2        20
     age  sex  agesexgr
85    42    2        18
87    43    2        18
89    44    2        18
91    45    2        18
93    46    2        18
95    47    2        18
97    48    2        18
99    49    2        18
101   50    2        18
103   51    2        18


,age,sex,agesexgr
120,60,1,19
121,60,2,20
122,61,1,19
123,61,2,20
124,62,1,19
125,62,2,20
126,63,1,19
127,63,2,20
128,64,1,19
129,64,2,20
